# DATA PROJECT: A dive into the Danish private sector

**TABLE OF CONTENTS**<a id='toc0_'></a>    
- [1. Project description](#toc1_)    
- [2. Imports and set magics](#toc2_) 
- [3. Read, clean and explore data](#toc3_)
    - 3.1. [Dataset GF02 - active firms given time, industry, size and region](#toc3_1_)
    - 3.2. [Dataset DEMO4 - start-ups and bankruptcies over time and for different industries](#toc3_2_)  
- [4. Merging the datasets](#toc4_)
- [5. Analysis](#toc5_)
- [6. Concluding remarks](#toc6_)
   

## 1. <a id='toc1_'></a>[Project description](#toc0_)

The idea behind the project is to...

## 2. <a id='toc2_'></a>[Imports and set magics](#toc0_)

In [36]:
# As we will use data from Statistics Denmark (Statistikbanken) we will be using the DstApi to access that data.
# If you previously have not installed it through github, the hashtag below should be deleted and the "%pip..." line should be run.
# More info the DstApi can be found by following the link below to the githhub page explaining the Api.

#%pip install git+https://github.com/alemartinello/dstapi


In [37]:
# We import different packages for the further analysis - among these the DstApi that has been installed via the code above.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})
import datetime
import ipywidgets as widgets
from matplotlib_venn import venn2
from dstapi import DstApi

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 3. <a id='toc3_'></a>[Read, clean and explore data](#toc0_)

### 3.1. <a id='toc3_1_'></a>[Dataset GF02 - active firms given time, industry, size and region](#toc0_)

#### Importing the dataset

In [38]:
gf02_api = DstApi('GF02') 

In [39]:
tabsum = gf02_api.tablesummary(language='en')
display(tabsum)

Table GF02: General enterprise statistics by Municipality, Industry (DB07 10- 19- and 127 grouping and detailed), enterprise size (full time equivalents) and time
Last update: 2023-05-05T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,KOMK,118,000,All Denmark,12,Outside from Denmark,False
1,BRANCHEDB0710TIL127,902,TOT,TOT Industry total,999999,999999 Activity not stated,False
2,FIRMSTR,6,TOT,"Total, all enterprises",103,250 employees and more,False
3,Tid,15,2007,2007,2021,2021,True


In [40]:
# The available values for a each variable: 
for variable in tabsum['variable name']:
    print(variable+':')
    display(gf02_api.variable_levels(variable, language='en'))

KOMK:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,101,Copenhagen
4,147,Frederiksberg
...,...,...
113,820,Vesthimmerlands
114,851,Aalborg
115,099,Region unknown
116,098,Municipality unknown


BRANCHEDB0710TIL127:


,id,text
0,TOT,TOT Industry total
1,1,"1 Agriculture, forestry and fishing"
2,A,"A Agriculture, forestry and fishing"
3,01000,01000 Agriculture and horticulture
4,011100,"011100 Growing of cereals (except rice), legum..."
...,...,...
897,990000,990000 Activities of extraterritorial organiza...
898,11,11 Activity not stated
899,X,X Activity not stated
900,99999,99999 Activity not stated


FIRMSTR:


,id,text
0,TOT,"Total, all enterprises"
1,0000,No employed
2,010,Less than 10 employed
3,101,10-49 employees
4,102,50-249 employees
5,103,250 employees and more


Tid:


,id,text
0,2007,2007
1,2008,2008
2,2009,2009
3,2010,2010
4,2011,2011
5,2012,2012
6,2013,2013
7,2014,2014
8,2015,2015
9,2016,2016


In [41]:
params = gf02_api._define_base_params(language='en')
params

{'table': 'gf02',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KOMK', 'values': ['*']},
  {'code': 'BRANCHEDB0710TIL127', 'values': ['*']},
  {'code': 'FIRMSTR', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [42]:
params_gf02 = {'table': 'gf02',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KOMK', 'values': ['000']}, #we look at companies in all of Denmark
  {'code': 'BRANCHEDB0710TIL127', 'values': ['TOT','1','2','3','4','5','6','7','8','9','10','11']}, #we look at all industries in Denmark (1-11) and the total
  {'code': 'FIRMSTR', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [43]:
gf02_all = gf02_api.get_data(params=params_gf02)
gf02_all

,KOMK,BRANCHEDB0710TIL127,FIRMSTR,TID,INDHOLD
0,All Denmark,"1 Agriculture, forestry and fishing",No employed,2007,30932
1,All Denmark,"1 Agriculture, forestry and fishing",Less than 10 employed,2007,9129
2,All Denmark,"1 Agriculture, forestry and fishing",10-49 employees,2007,369
3,All Denmark,"1 Agriculture, forestry and fishing",50-249 employees,2007,21
4,All Denmark,"1 Agriculture, forestry and fishing",250 employees and more,2007,2
...,...,...,...,...,...
1039,All Denmark,TOT Industry total,Less than 10 employed,2021,154138
1040,All Denmark,TOT Industry total,10-49 employees,2021,19594
1041,All Denmark,TOT Industry total,50-249 employees,2021,3945
1042,All Denmark,TOT Industry total,250 employees and more,2021,870


#### Cleaning and renaming the dataset

In [44]:
gf02_size = gf02_all.drop(columns=['KOMK', 'BRANCHEDB0710TIL127'])
gf02_size

,FIRMSTR,TID,INDHOLD
0,No employed,2007,30932
1,Less than 10 employed,2007,9129
2,10-49 employees,2007,369
3,50-249 employees,2007,21
4,250 employees and more,2007,2
...,...,...,...
1039,Less than 10 employed,2021,154138
1040,10-49 employees,2021,19594
1041,50-249 employees,2021,3945
1042,250 employees and more,2021,870


In [45]:
gf02_size_rename = gf02_size.rename(columns = {'FIRMSTR':'size', 'TID':'year', 'INDHOLD':'#companies'})
gf02_size_rename

,size,year,#companies
0,No employed,2007,30932
1,Less than 10 employed,2007,9129
2,10-49 employees,2007,369
3,50-249 employees,2007,21
4,250 employees and more,2007,2
...,...,...,...
1039,Less than 10 employed,2021,154138
1040,10-49 employees,2021,19594
1041,50-249 employees,2021,3945
1042,250 employees and more,2021,870


In [ ]:
gf02_size_wide = pd.pivot(gf02_size_rename, index='size', columns='year', values='#companies')
gf02_size_wide = gf02_size_wide.reindex(['No employed', 'Less than 10 employed', '10-49 employees', '50-249 employees', '250 employees and more', 'Total, all enterprises'])
gf02_size_wide

In [ ]:
def plot_gf02_size(df, size, industry): 
    filtered_df = df[(df['size'] == size) & (df['industry'] == industry)]
    ax=filtered_df.plot(x='year', y='#companies', style='-o', legend=False)

In [ ]:
widgets.interact(plot_gf02_size, 
    df = widgets.fixed(gf02_size),
    industry=widgets.Dropdown(description='Industry', options=gf02_size['industry'].unique(), value='1 Agriculture, forestry and fishing'),
    size = widgets.Dropdown(description='Size', options=gf02_size['size'].unique(), value='Total, all enterprises'),
);

In [ ]:
gf02_size_wide.describe()

In [ ]:
#Call function from py-file
#var_keep = ['region', 'All Denmark']
#gf02_filtered = dataproject.keep_regs(gf02_all, var_keep)

In [ ]:
#I = gf02_all.regions.str.contains('Region')
#I |= gf02_all.regions.str.contains('All Denmark')
#gf02_all.loc[I, :]

In [ ]:
#gf02_reduce = gf02_all.loc[I == True] # drop everything else
#gf02_reduce.reset_index(inplace = True, drop = True) # Drop old index too. If the drop is not used, python will create another dataset with the old index.
#gf02_reduce.iloc[0:153,:]
#gf02_reduce

#### Exploring the dataset

In [ ]:
gf02_size_plot1 = gf02_size.loc[gf02_size['size'] == 'Total, all enterprises', :]

# Plot the content of the data frame
gf02_size_plot1.plot(x='year',y='#companies',legend="total companies in Denmark");

In [ ]:
def plot_gf02_size(df, size): 
    I = df['size'] == size
    ax=df.loc[I,:].plot(x='year', y='#companies', style='-o', legend=False)

In [ ]:
widgets.interact(plot_gf02_size, 
    df = widgets.fixed(gf02_size),
    size = widgets.Dropdown(description='Size', 
                            options=gf02_size['size'].unique(), 
                            value='Total, all enterprises')
);

### 3.2. <a id='toc3_2_'></a>[Dataset DEMO4 - start-ups and bankruptcies over time and for different industries](#toc0_)

#### Importing the dataset

As we are using another dataset from Statistics Denmark, we apply the same approach as described with the previous dataset (GF02).

In [ ]:
demo4_api = DstApi('DEMO4')

In [ ]:
tabsum2 = demo4_api.tablesummary(language='en')
display(tabsum2)

In [ ]:
# The available values for a each variable: 
for variable in tabsum2['variable name']:
    print(variable+':')
    display(demo4_api.variable_levels(variable, language='en'))

In [ ]:
params_demo4 = demo4_api._define_base_params(language='en')
params_demo4

In [ ]:
params_demo4 = {'table': 'demo4',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'REGION', 'values': ['*']},
  {'code': 'BRANCHEDB0710', 'values': ['*']},
  {'code': 'MÆNGDE4', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [ ]:
demo4_all = demo4_api.get_data(params=params_demo4)
demo4_all.head(5)

#### Cleaning and renaming the dataset

In [ ]:
demo4_all.rename(columns = {'REGION':'regions','BRANCHEDB0710':'Industry', 'MÆNGDE4': 'X', 'TID':'Time', 'INDHOLD':'Total'}, inplace=True)
demo4_all.head()

In [ ]:
I = demo4_all.regions.str.contains('regions')
I |= demo4_all.regions.str.contains('All Denmark')
demo4_all.loc[I, :]

In [ ]:
demo4_reduce = demo4_all.loc[I == True] # drop everything else
demo4_reduce.reset_index(inplace = True, drop = True) # Drop old index too. If the drop is not used, python will create another dataset with the old index.
demo4_reduce.iloc[0:214,:]
demo4_reduce

In [ ]:
demo4_reduce.describe()

In [ ]:
I = demo4_all.regions.str.contains('Region')
demo4_all.loc[I, :]

In [ ]:
demo4_reduced = demo4_all.loc[I == False] # keep everything else
demo4_reduced.head(10)

In [ ]:
demo4_reduced.reset_index(inplace = True, drop = True) # Drop old index too. If the drop is not used, python will create another dataset with the old index.
demo4_reduced.iloc[0:214,:]
demo4_reduced

#### Exploring the dataset

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

## 4. <a id='toc4_'></a>[Merging the datasets](#toc0_)

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

## 5. <a id='toc5_'></a>[Analysis](#toc0_)

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

## 6. <a id='toc6_'></a>[Concluding remarks](#toc0_)

ADD CONCISE CONLUSION.